In [3]:
import damask
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
resultfile_name = 'Polycrystal_4_4x4x4_load_material.hdf5'

# Example: Stress-strain curve for tensionX

In [ ]:
result = damask.Result(resultfile_name)
result.add_stress_Cauchy()

grid   = damask.GeomGrid.load(fn_vti) # get back my grid
# loop over all increments and store P(1,1) per grain and avg(F(1,1))
data = {g:pd.DataFrame() for g in grains}
for inc in result.get(['F','P']).values():
    P = inc['P']
    F = inc['F']
    for g in grains:
        points = grid.material.flatten(order='F')==g
        P_11 = P[points,0,0].flatten()
        F_11 = np.broadcast_to(np.average(F[:,0,0]),P_11.shape)
        x = pd.DataFrame({'F_11':F_11,'P_11':P_11})
        data[g] = pd.concat((data[g],x),ignore_index=True)

# Example: Stress-Strain Density plot for each grain 

In [ ]:
result = damask.Result(resultfile_name)
last_inc = result.view(increments=-1)
last_inc = result.view(homogenizations=False)
df = pd.DataFrame({'F_11':result.get('F')[:,0,0],
                   'P_11':result.get('P')[:,0,0]})
_ = df.plot.hexbin(x='F_11',y='P_11',gridsize=20,
                   cmap=damask.Colormap.from_predefined('strain'))

# Example: r-value calculation

In [1]:
result = damask.Result(resultfile_name)
result.add_strain('F_p','U')
result.add_equivalent_Mises('epsilon_U^0.0(F_p)')
epsilon_avg = np.array([np.average(eps,0) for eps in result.get('epsilon_U^0.0(F_p)').values()])
r = epsilon_avg[:,1,1]/epsilon_avg[:,2,2]
s = np.array([np.average(strain) for strain in result.get('epsilon_U^0.0(F_p)_vM').values()])
plt.plot(s[10:],r[10:])
plt.ylabel('r-value')
plt.xlabel('plastic vM strain')
plt.axis([0, 0.2, 0, 1])
plt.show()